In [1]:
import csv
datafile = 'data/write_in_responses.tsv'

lines = []
with open(datafile) as f:
    reader = csv.reader(f, delimiter='\t')
    header = next(reader)
    for line in reader:
        if len(line) != 13:
            print(len(line))
        lines.append(line)       

In [2]:
import re
pattern = 'Question ([0-9]+) - (.+)'
question_numbers = []
questions = []
for elem in header:
    matched = re.findall(pattern, elem)[0]
    question_numbers.append(matched[0])
    questions.append(int(matched[1])
    
print(question_numbers)
print(questions)

['7', '9', '10', '11', '12', '13', '14', '17', '18', '19', '20', '21', '23']
['If you do have an accessibility need(s) or impairment(s), how do you manage the impact it has on how you receive information in poster or oral presentations? ', 'When attending a poster session, what is your goal in receiving information? Please rate your priority in learning about each of the of the following aspects of the study on a scale of 1 to 5:', 'Traditional layout: Please rate how this poster design impacts your ability to gain information on a scale from 1 to 5. ', '#BetterPoster layout: Please rate how this poster design impacts your ability to gain information on a scale from 1 to 5. ', 'Infographic layout eaxample 1: Please rate how this poster design impacts your ability to gain information on a scale from 1 to 5.', 'Infographic layout eaxample 2: Please rate how this poster design impacts your ability to gain information on a scale from 1 to 5. ', 'What improvements, if any, would you like to

In [3]:
from collections import defaultdict, namedtuple

Response = namedtuple('Response', 'row_id text')

question_answers = defaultdict(list)
# may want to keep the information from the people to correlate with 
# the data in the other questions (e.g., if the person had a visual 
# limitation, are they more likely to request visual accomodation?)
for row_id, line in enumerate(lines):
    for i, resp in enumerate(line):
        if resp:
            question_answers[question_numbers[i]].append(Response(row_id=row_id, text=resp))

question_answers.keys()

dict_keys(['19', '20', '10', '11', '14', '18', '21', '12', '13', '23', '7', '17', '9'])

In [4]:
from collections import Counter
import string

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stops = stops = {
    "a",
    "about",
    "again",
    "am",
    "an",
    "are",
    "and",
    "as",
    "at",
    "be",
    "both",
    "but",
    "by",
    "can",
    "could",
    "did",
    "do",
    "does",
    "doing",
    "during",
    "each",
    "for",
    "from",
    "further",
    "had",
    "has",
    "have",
    "having",
    "he",
    "he'd",
    "he'll",
    "he's",
    "her",
    "here",
    "here's",
    "hers",
    "herself",
    "him",
    "himself",
    "his",
    "how",
    "how's",
    "i",
    "i'd",
    "i'll",
    "i'm",
    "i've",
    "if",
    "in",
    "into",
    "is",
    "it",
    "it's",
    "its",
    "itself",
    "let's",
    "me",
    "more",
    "most",
    "my",
    "myself",
    "no",
    "nor",
    "not",
    "of",
    "or",
    "other",
    "ought",
    "our",
    "ours	ourselves",
    "out",
    "own",
    "shan't",
    "she",
    "she'd",
    "she'll",
    "she's",
    "should",
    "so",
    "some",
    "such",
    "than",
    "that",
    "that's",
    "the",
    "their",
    "theirs",
    "them",
    "themselves",
    "then",
    "there",
    "there's",
    "these",
    "they",
    "they'd",
    "they'll",
    "they're",
    "they've",
    "this",
    "those",
    "to",
    "too",
    "up",
    "very",
    "was",
    "wasn't",
    "we",
    "we'd",
    "we'll",
    "we're",
    "we've",
    "were",
    "weren't",
    "what",
    "what's",
    "when",
    "when's",
    "where",
    "where's",
    "which",
    "while",
    "who",
    "who's",
    "whom",
    "why",
    "why's",
    "with",
    "won't",
    "would",
    "you",
    "you'd",
    "you'll",
    "you're",
    "you've",
    "your",
    "yours",
    "yourself",
    "yourselves",
}



def ngram_counter(n: int, texts: list[str], lower: bool = True) -> Counter[str]:
    c = Counter()
    for text in texts:
        if lower:
            text = text.lower()
        for s in sliding_ngram_windows(text, n):
            s = ' '.join(s)
            c[s] += 1
    return c


def prune_stop_tokens(toks: list[str]) -> list[str]:
    return [t for t in toks if t.lower() not in stops]


def sliding_ngram_windows(s: str, n: int, keep_punct: bool = False) -> list[list[str]]:
    toks = strip_and_split(s) if not keep_punct else s.split()
    pruned = prune_stop_tokens(toks)
    return [pruned[i : i + n] for i in range(0, len(pruned) - n + 1)]


def strip_and_split(s: str) -> list[str]:
    return strip_punct(s).split()


def strip_punct(s: str) -> str:
    return s.translate(str.maketrans('', '', string.punctuation))


In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import nltk
import numpy as np

from sklearn import metrics

embedder = SentenceTransformer('all-MiniLM-L6-v2')


def create_sentence_corpus(passages):
    corpus = []
    for text in passages:
        for sent in nltk.sent_tokenize(text):
            corpus.append(sent)
    return corpus


def embed_corpus(corp):
    embs = embedder.encode(corp)
    # Normalize the embeddings to unit length
    embs = embs /  np.linalg.norm(embs, axis=1, keepdims=True)
    return embs


def agg_cluster(sents, embeds, dist_thresh, linkage):
    clustering_model = AgglomerativeClustering(n_clusters=None, metric='cosine', linkage=linkage, distance_threshold=dist_thresh)
    clustering_model.fit(embeds)
    cluster_assignment = clustering_model.labels_

    clustered_sentences = defaultdict(list)
    clustered_embeddings = defaultdict(list)
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        if True:
            clustered_sentences[cluster_id].append(sents[sentence_id])
            clustered_embeddings[cluster_id].append(embeds[sentence_id])
    
    print(metrics.silhouette_score(embeds, cluster_assignment, metric='cosine'))
    
    return clustered_sentences, clustered_embeddings

def mean_shift_cluster(sents, embeds):
    from sklearn.cluster import MeanShift
    
    clustering_model = MeanShift(cluster_all=False)
    clustering_model.fit(embeds)
    cluster_assignment = clustering_model.labels_
    
    clustered_sentences = defaultdict(list)
    clustered_embeddings = defaultdict(list)
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        if True:
            clustered_sentences[cluster_id].append(sents[sentence_id])
            clustered_embeddings[cluster_id].append(embeds[sentence_id])
    
    print(metrics.silhouette_score(embeds, cluster_assignment, metric='cosine'))
    
    return clustered_sentences, clustered_embeddings
    


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial

def avg_cosine_sim(cluster_embeddings):
    sims = []
    if len(cluster_embeddings) == 1:
        return 0.0
    for i, emb in enumerate(cluster_embeddings):
        for j, emb2 in enumerate(cluster_embeddings):
            if i != j:
                sims.append(cosine_similarity(np.reshape(emb, (1, -1)), np.reshape(emb2, (1, -1))))
    return sum(sims) / len(sims)

def score_clusters(clustered_sentences, clustered_embeddings):
    scored = []
    for cluster_id, embs in clustered_embeddings.items():
        scored.append((cluster_id, avg_cosine_sim(embs), clustered_sentences[cluster_id]))
    return scored

def display_scored_clusters(scored_clusters):
    for (cluster_id, score, cluster) in sorted(scored_clusters, key=lambda tup: tup[1], reverse=True):
        print("Cluster ", cluster_id+1)
        print("Avg cosine sim: ", score)
        print(cluster)
        print("")

def cluster(corpus, embeddings, dist_thresh, display=True, linkage='complete', cluster_method='agg'):
    cluster_func = None
    if cluster_method == 'agg':
        cluster_func = partial(agg_cluster, dist_thresh=dist_thresh, linkage=linkage)
    elif cluster_method == 'meanshift':
        cluster_func = mean_shift_cluster
        
    
    clustered_sentences, clustered_embeddings = cluster_func(sents=corpus, embeds=embeddings)
    scored = score_clusters(clustered_sentences, clustered_embeddings)
    if display:
        display_scored_clusters(scored)
    return scored


In [2]:
from pathlib import Path
from transformers import pipeline
from tqdm import tqdm

summarizer = pipeline("summarization", model='knkarthick/MEETING_SUMMARY')

def summarize(text):
    num_toks = len(text.split(' '))
    summary = summarizer(text, max_length=30)
    return summary

def create_summaries(scored):
    summaries = []
    sorted_clusters = sorted(scored, key=lambda tup: tup[1], reverse=True)
    for i in tqdm(range(0, len(scored)), desc ="Summarizing clusters"):
        cluster_sents = sorted_clusters[i][2]
        try:
            summaries.append(summarize(' '.join(cluster_sents)))
        except:
            summaries.append('ERROR: was unable to summarize')
    return summaries
    
def export_review_file(fn, num_responses, scored, summaries):
    with open(fn, 'w') as f:
        print(f'Number of responses to the question: {num_responses}', file=f)
        print(f'Number of clusters found: {len(scored)}', file=f)
        more_than_one = [x for x in scored if len(x[2]) > 1]
        print(f'Number of clusters with more than one sentence...: {len(more_than_one)}\n', file=f)
        for i, (cluster_id, score, cluster) in enumerate(sorted(scored, key=lambda tup: tup[1], reverse=True)):
            print(f"Cluster {cluster_id+1}\n", file=f)
            print(f"Avg cosine similarity of this cluster: {score}", file=f)
            print(f"Number of sentences in this cluster: {len(cluster)}\n", file=f)
            print("Cluster sentences, in no particular order:", file=f)
            for sent in cluster:
                print(f'\t{sent}', file=f)
            summary = summaries[i]
            print(f"\nML Cluster Summary: {summary}", file=f)
            print("\n\n--------------------------------\n\n", file=f)




/Users/sharpr1/.virtualenvs/lex-ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
def sentence_level_analysis(question_number, corpus, dist_thresh, linkage, cluster_method='agg'):
    num_responses = len(corpus)
    corpus = create_sentence_corpus(corpus)
    corpus_embeddings = embed_corpus(corpus)

    scored_curr = cluster(corpus, corpus_embeddings, dist_thresh, display=False, linkage=linkage, cluster_method=cluster_method)
    summaries_curr = create_summaries(scored_curr)             

    outdir = Path('/Users/sharpr1/data/accessibility') 
    outfile = outdir / f'clusters_for_question_{question_number}_sentence_level_{cluster_method}_{linkage}_{dist_thresh}.txt'
    export_review_file(outfile, num_responses, scored_curr, summaries_curr)

In [10]:
question_answers.keys()

dict_keys(['19', '20', '10', '11', '14', '18', '21', '12', '13', '23', '7', '17', '9'])

In [27]:
for question_number, responses in question_answers.items():
    q_texts = [r.text for r in responses]
    print("Processing responses for question", question_number)
    sentence_level_analysis(question_number, q_texts, 0.7, 'complete')


Processing responses for question 19
0.12319375


Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [01:29<00:00,  1.79s/it]


Processing responses for question 20
0.1191813


Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 56/56 [01:30<00:00,  1.61s/it]


Processing responses for question 10
0.10961497


...

Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 28/28 [00:42<00:00,  1.53s/it]


Processing responses for question 17
0.16059878


Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.37s/it]


Processing responses for question 9
0.11876916


Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 15/15 [00:25<00:00,  1.72s/it]


In [29]:
for question_number, responses in question_answers.items():
    q_texts = [r.text for r in responses]
    print("Processing responses for question", question_number)
    sentence_level_analysis(question_number, q_texts, 0.9, 'complete')

Processing responses for question 19
0.11434844


Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 19/19 [00:43<00:00,  2.26s/it]


Processing responses for question 20
0.07048876


Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 18/18 [00:40<00:00,  2.24s/it]


Processing responses for question 10
0.1052975


...

Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:21<00:00,  1.80s/it]


Processing responses for question 17
0.13288675


Summarizing clusters: 100%|█████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:10<00:00,  1.74s/it]


Processing responses for question 9
0.103431344


Summarizing clusters: 100%|█████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.97s/it]
